# STEP 0: Install dependencies



In [6]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

# STEP 1: Imports and secure API key input

In [ ]:
import os
import getpass #for securing and enables for getting api as password. anything which is passed inside getpass will become a password
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ['GOOGLE_API_KEY']=getpass.getpass('Please enter your Gemini API key: ')

Please enter your Gemini API key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [51]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.2)
#temperature controls the randomness, higher temp it will become creative
#if temperature is low, then ai will give the same output

# STEP 3: Node to ask user for symptom

In [10]:
def get_symptom(state:dict)->dict:
  symptom=input("Welcome to XYZ Hospital, Please enter your symptom: ")
  state["symptom"]=symptom
  return state

# STEP 4: Node to classify the symptom

In [40]:
from langchain_core.messages import HumanMessage

def classify_symptom(state: dict) -> dict:
    prompt = (
        "You are a helpful medical agent. Classify the symptom below into one of these categories:\n"
        "- General\n- Emergency\n- Mental Health\n\n"
        f"Symptom: {state['symptom']}\n\n"
        "Respond only with one word: General, Emergency, or Mental Health.\n"
        "Example: input: I have fever → Output: General"
    )

    response = llm.invoke([HumanMessage(content=prompt)])
    category = response.content.strip()

    print(f"LLM Classifies the symptom as: {category}")
    state["category"] = category
    return state


# STEP 5: Router logic to route to the correct node


In [41]:
def symptom_router(state: dict) -> str:
    cat = state["category"].lower().replace(" ", "-")

    if "general" in cat:
        return "general"
    elif "emergency" in cat:
        return "emergency"
    elif "mental-health" in cat:
        return "mental-health"
    else:
        return "general"  # Default fallback route


# STEP 6: Category-specific response nodes


In [42]:
def general_node(state: dict) -> dict:
    state["answer"] = (
        f"'{state['symptom']}': Seems like a general issue. "
        "Directing you to the General Ward for consultation."
    )
    return state

def emergency_node(state: dict) -> dict:
    state["answer"] = (
        f"'{state['symptom']}': This is classified as a **Medical Emergency**. "
        "Seeking immediate assistance."
    )
    return state

def mental_health_node(state: dict) -> dict:
    state["answer"] = (
        f"'{state['symptom']}': This may be related to **mental health**. "
        "Please talk to our mental health consultant."
    )
    return state


# STEP 7: Build LangGraph


In [47]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

class GraphState(TypedDict):
    symptom: str
    category: str
    answer: str

builder = StateGraph(GraphState)

# Define nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental-health", mental_health_node)

# Define edges
builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental-health": "mental-health"
})
builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental-health", END)




# STEP 8: Compile and invoke the graph


In [48]:
# Compile & show graph
graph = builder.compile()
mermaid = graph.get_graph()
print("Mermaid Graph:")
print(mermaid)

Mermaid Graph:
Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'get_symptom': Node(id='get_symptom', name='get_symptom', data=get_symptom(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'classify': Node(id='classify', name='classify', data=classify(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'general': Node(id='general', name='general', data=general(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'emergency': Node(id='emergency', name='emergency', data=emergency(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'mental-health': Node(id='mental-health', name='mental-health', data=mental-health(tags=None, recurse=Tr

In [52]:
final_state = graph.invoke({})
print("Final output:")
print(final_state["answer"])


Welcome to XYZ Hospital, Please enter your symptom: cough and cold
LLM Classifies the symptom as: General
Final output:
'cough and cold': Seems like a general issue. Directing you to the General Ward for consultation.
